# Two-phase flow model

In this tutorial we present how to solve the two-phase flow model with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknowns are the velocity $q_\alpha$ and the pressure $p_\alpha$ of both phases.

### Solution strategy

The system is given by:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\phi \partial_t S(p_n, p_w) + \nabla \cdot q_w = f_w \\
- \phi \partial_t S(p_n, p_w) + \nabla \cdot q_n = f_n \\
q_w = -\frac{k_i}{\mu_w} k_w \left(S(p_n, p_w)\right) \nabla (p_w - \rho_w g) \\
q_n = -\frac{k_i}{\mu_n} k_n \left(1 - S(p_n, p_w)\right) \nabla (p_n - \rho_n g)
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

Let $S^n = \phi S(p_n,p_w)$, $K_w = \frac{k_i}{\mu_w} k_w \left(S(p_n, p_w)\right)$, $K_n = \frac{k_i}{\mu_n} k_n \left(1 - S(p_n, p_w)\right)$  and $z_\alpha = \rho_\alpha g$, 
Such that the system of equations read: 
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\partial_t S + \nabla \cdot q_w = f_w \\
- \partial_t S + \nabla \cdot q_n = f_n \\
q_w = -K_w \nabla (p_w - z_w) \\
q_n = -K_n \nabla (p_n - z_n)
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$



We will use the Backward Euler discretization scheme to discretize the equation in time:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\frac{S^{n} - S^{n-1}}{\tau} + \nabla \cdot q_w^{n} = f_w^{n} \\
- \frac{S^{n} - S^{n-1}}{\tau} + \nabla \cdot q_n^{n} = f_n^{n} \\
q_w^{n} = -K_w^{n} \nabla (p_w^{n} - \rho_w g) \\
q_n^{n} = -K_n^{n} \nabla (p_n^{n} - \rho_n g)
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$


With the time discretization of the abbreviations being $S^n = \phi S(p_n^{n},p_w^{n})$, $K_w^{n} = \frac{k_i}{\mu_w} k_w \left(S(p_n^{n}, p_w^{n})\right)$, $K_n^{n} = \frac{k_i}{\mu_n} k_n \left(1 - S(p_n^{n}, p_w^{n})\right)$ and $z_\alpha = \rho_\alpha g$.

We will use the L-scheme to deal with the non-linearities in $S(p_n, p_w)$, and letting  $q_\alpha^{n,i} = -K_\alpha^{n,i-1} \nabla (p_\alpha^{n,i} - z_\alpha)$, obtaining:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
S^{n,i-1} + L_w (p_w^{n,i} - p_w^{n,i-1}) - S^{n-1} + \tau \nabla \cdot q_w^{n,i} = \tau f_w^{n} \\
- S^{n,i-1} - L_n (p_n^{n,i} - p_n^{n,i-1}) + S^{n-1} + \tau \nabla \cdot q_n^{n,i} = \tau f_n^{n} \\
q_w^{n,i} = -K_w^{n,i-1} \nabla (p_w^{n,i} - z_w) \\
q_n^{n,i} = -K_n^{n,i-1} \nabla (p_n^{n,i} - z_n)
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

With the linearized abbreviations being $S^{n,i} = \phi S(p_n^{n,i},p_w^{n,i})$, $K_w^{n,i} = \frac{k_i}{\mu_w} k_w \left(S(p_n^{n,i}, p_w^{n,i})\right)$ and $K_n^{n,i} = \frac{k_i}{\mu_n} k_n \left(1 - S(p_n^{n,i}, p_w^{n,i})\right)$.



The weak formulation of the problem then reads:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
\langle S^{n,i-1}, v_w \rangle_\Omega + L_w \langle p_w^{n,i}, v_w \rangle_\Omega - L_w \langle p_w^{n,i-1}, v_w \rangle_\Omega - \langle S^{n-1}, v_w \rangle_\Omega - \tau \langle q_w^{n,i}, \nabla v_w \rangle_\Omega + \tau \langle q_w^{n,i} \cdot \bold{n}, v_w \rangle_\Gamma = \tau \langle f_w^{n}, v_w \rangle_\Omega \\
- \langle S^{n,i-1}, v_n \rangle_\Omega - L_n \langle p_n^{n,i}, v_n \rangle_\Omega + L_n \langle p_n^{n,i-1}, v_n \rangle_\Omega + \langle S^{n-1}, v_n \rangle_\Omega - \tau \langle q_n^{n,i}, \nabla v_n \rangle_\Omega + \tau \langle q_n^{n,i} \cdot \bold{n}, v_n \rangle_\Gamma = \tau \langle f_n^{n}, v_n \rangle_\Omega \\
\langle q_w^{n,i}, w_w \rangle_\Omega = - K_w^{n,i-1}(- \langle p_w^{n,i}, \nabla w_w \rangle_\Omega + \langle p_w^{n,i}, w_w \cdot \bold{n} \rangle_\Gamma - \langle \nabla z_w, w_w \rangle_\Omega) \\
\langle q_n^{n,i}, w_n \rangle_\Omega = - K_n^{n,i-1}(- \langle p_n^{n,i}, \nabla w_n \rangle_\Omega + \langle p_n^{n,i}, w_n \cdot \bold{n} \rangle_\Gamma - \langle \nabla z_n, w_n \rangle_\Omega) \\
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

Written in terms of unknowns:  
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 

L_w \langle p_w^{n,i}, v_w \rangle_\Omega - \tau \langle q_w^{n,i}, \nabla v_w \rangle_\Omega = \langle S^{n-1}, v_w \rangle_\Omega - \langle S^{n,i-1}, v_w \rangle_\Omega + L_w \langle p_w^{n,i-1}, v_w \rangle_\Omega + \tau \langle f_w^{n}, v_w \rangle_\Omega - \tau \langle q_w^{n,i} \cdot \bold{n}, v_w \rangle_\Gamma \\

L_n \langle p_n^{n,i}, v_n \rangle_\Omega + \tau \langle q_n^{n,i}, \nabla v_n \rangle_\Omega = \langle S^{n-1}, v_n \rangle_\Omega - \langle S^{n,i-1}, v_n \rangle_\Omega + L_n \langle p_n^{n,i-1}, v_n \rangle_\Omega - \tau \langle f_n^{n}, v_n \rangle_\Omega + \tau \langle q_n^{n,i} \cdot \bold{n}, v_n \rangle_\Gamma \\

\langle \bar{K_w}^{n,i-1} q_w^{n,i}, w_w \rangle_\Omega - \langle p_w^{n,i}, \nabla w_w \rangle_\Omega = \langle \nabla z_w, w_w \rangle_\Omega - \langle p_w^{n,i}, w_w \cdot \bold{n} \rangle_\Gamma \\

\langle \bar{K_n}^{n,i-1} q_n^{n,i}, w_n \rangle_\Omega - \langle p_n^{n,i}, \nabla w_n \rangle_\Omega = \langle \nabla z_n, w_n \rangle_\Omega - \langle p_n^{n,i}, w_n \cdot \bold{n} \rangle_\Gamma \\

\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$

The space discretization is perfomed using FEM ($RT_{0}$ elements for ${q}$ and $P_0$ for ${p}$). The resulting linear system that must be solved for each step and each iteration is:


We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

The test case presented is based on the following article [10.1007/s10596-016-9566-3](https://link.springer.com/article/10.1007/s10596-016-9566-3).

### Import and parameters

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
import sympy as sp

import porepy as pp
import pygeon as pg

In [2]:
# Set the maximum number of iterations of the non-linear solver
K = 50

# L-scheme parameter
L = 3.501e-2

# Set the mesh refinment
N = 20

# Set the number of steps (excluding the initial condition)
num_steps = 9

# Simulation time length
T = 9 / 48

# Time switch conditions (for the boundary condition)
dt_D = 1 / 16

# Fluid density
rho_w = 1000
rho_n = 1000

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-6
rel_tol = 1e-6

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = "output_evolutionary"

In [3]:
# Van Genuchten model parameters ( relative permeability model )
theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2

m = 1 - 1 / n

In [4]:
# Time step
dt = (T - 0) / num_steps

### $\theta$ and $K$

In order to evaluate analytically both $\theta$ and $K$, we will make use of the python package $\textit{sympy}$, that allow us to write down a mathematical expression in symbolic form (and that can also be used to compute their derivatives).

In [5]:
# Symbolic psi
psi_var = sp.Symbol("psi", negative=True)

# Symbolic Theta
theta_expression = theta_r + (theta_s - theta_r) / (1 + (-alpha * psi_var) ** n) ** m
effective_saturation = (theta_expression - theta_r) / (theta_s - theta_r)

# Symbolic Conductivity K
hydraulic_conductivity_expression = (
    K_s
    * (effective_saturation**0.5)
    * (1 - (1 - effective_saturation ** (1 / m)) ** m) ** 2
)

In [6]:
# Theta lambda
theta_lambda = sp.lambdify(psi_var, theta_expression, "numpy")

# Conductivity tensor lambda
conductivity_lambda = sp.lambdify(psi_var, hydraulic_conductivity_expression, "numpy")

In [7]:
# Actual (and final) theta function
def theta(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * theta_s
    res[mask] = theta_lambda(psi[mask])

    return res

In [8]:
# Actual (and final) theta function
def conductivity(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * K_s
    res[mask] = conductivity_lambda(psi[mask])

    return res

### Domain preparation

In [9]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2 * N, 3 * N], [2, 3])

# Convert it to a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])
pg.convert_from_pp(mdg)

In [10]:
key = "flow"

# Collection of boundary conditions
bc_value = []
bc_essential = []

# Initial pressure
initial_pressure = []

# Discretizations for q and \psi
RT0 = pg.RT0(key)
P0 = pg.PwConstants(key)

In [11]:
# Gravity. The y-direction is set instead of the z-direction because we are in the 2D case
g_func = lambda x, t: np.array([0, -1, 0])

# Initial pressure function
initial_pressure_func = lambda x: 1 - x[1]

In [12]:
# Fake loop to extract the grid and its data (i.e. conductivity tensor)
for subdomain, data in mdg.subdomains(return_data=True):
    # Prepare the gravity term Z by firstly projecting g_func into the RT0 
    # space and then by multiplying it by the RT0 mass matrix
    g_proj = RT0.interpolate(
        subdomain, 
        lambda x: g_func(x, 0)
    )
    gravity = RT0.assemble_mass_matrix(subdomain) @ g_proj

    # Prepare the inital pressure term by interpolating initial_pressure_func 
    # into the P0 space
    initial_pressure.append(P0.interpolate(subdomain, initial_pressure_func))

    # Get the boundary faces ids
    boundary_faces_indexes = subdomain.get_boundary_faces()

    # Gamma_D1 and Gamma_D2 boundary faces
    gamma_d1 = np.logical_and(
        subdomain.face_centers[0, :] > 0 - domain_tolerance,
        np.logical_and(
            subdomain.face_centers[0, :] < 1 + domain_tolerance,
            subdomain.face_centers[1, :] > 3 - domain_tolerance,
        ),
    )
    gamma_d2 = np.logical_and(
        subdomain.face_centers[0, :] > 2 - domain_tolerance,
        np.logical_and(
            subdomain.face_centers[1, :] > 0 - domain_tolerance,
            subdomain.face_centers[1, :] < 1 + domain_tolerance,
        ),
    )

    gamma_d = np.logical_or(gamma_d1, gamma_d2)

    # Gamma_N is the remaining part of the boundary
    gamma_n = gamma_d.copy()
    gamma_n[boundary_faces_indexes] = np.logical_not(gamma_n[boundary_faces_indexes])

    # Set the initial conductivity tensor in data (the actual saved 
    # tensor does not matter at this stage)
    pp.initialize_data(
        subdomain,
        data,
        key,
        {
            "second_order_tensor": pp.SecondOrderTensor(np.ones(subdomain.num_cells)),
        },
    )

    # Prepare the \hat{\psi} function
    def bc_gamma_d(x, t):
        if (
            x[0] > 2 - domain_tolerance
            and x[1] > 0 - domain_tolerance
            and x[1] < 1 + domain_tolerance
        ):
            res = 1 - x[1]
        elif (
            x[1] > 3 - domain_tolerance
            and x[0] > 0 - domain_tolerance
            and x[0] < 1 + domain_tolerance
        ):
            res = min(0.2, -2 + 2.2 * t / dt_D)
        else:
            res = 0

        return res

    # Add a lambda function that generates for each time instant the
    # (discretized) natural boundary conditions for the problem
    bc_value.append(
        lambda t: -RT0.assemble_nat_bc(subdomain, lambda x: bc_gamma_d(x, t), gamma_d)
    )

    # Set the essential boundary conditions (they will be enforced before 
    # solving the system)
    essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
    bc_essential = np.hstack((gamma_n, essential_pressure_dofs))

### Matrix assembly

In [13]:
# Psi mass matrix
M_psi = P0.assemble_mass_matrix(subdomain)

# B
B = -pg.cell_mass(mdg, P0) @ pg.div(mdg)

# Psi projection
proj_psi = P0.eval_at_cell_centers(subdomain)

# q projection
proj_q = RT0.eval_at_cell_centers(subdomain)

dof_psi, dof_q = B.shape

In [14]:
# Assemble initial solution
initial_solution = np.zeros(dof_q + dof_psi)
initial_solution[-dof_psi:] += np.hstack(initial_pressure)

# Final solution list. Each of its elements will be the solution at a specific instant
sol = [initial_solution]

In [15]:
# Assemble the fixed part of the right hand side (rhs)
fixed_rhs = np.zeros(dof_q + dof_psi)
fixed_rhs[:dof_q] = gravity

In [16]:
# Helper function to project a function evaluated in the cell center to FEM (scalar)
def project_psi_to_fe(to_project):
    return to_project * subdomain.cell_volumes

### Solving stage

In [17]:
# Delete the output directory, if it exisis
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [18]:
# Helper function to export the current_sol to a file
def export_solution(saver, current_sol, num_step):
    ins = list()

    ins.append(
        (
            subdomain,
            "cell_q",
            (proj_q @ current_sol[:dof_q]).reshape((3, -1)),
        )
    )
    ins.append((subdomain, "cell_p", proj_psi @ current_sol[dof_q : (dof_q + dof_psi)]))

    saver.write_vtu(ins, time_step=num_step)

In [19]:
# Prepare the porepy exporter and export the initial solution
saver = pp.Exporter(mdg, "richards", folder_name=output_directory)

export_solution(saver, current_sol=sol[-1], num_step=0)

In [20]:
# Time loop
for n in np.arange(num_steps):
    current_time = (n + 1) * dt
    print("Time " + str(round(current_time, 5)))

    # Rhs that changes with time (but not with k)
    time_rhs = fixed_rhs.copy()

    # Add the (natural) boundary conditions
    time_rhs[:dof_q] += np.hstack(list(cond(current_time) for cond in bc_value))

    # Add \Theta^n:
    # 1. Convert psi DOF to cell-wise values
    # 2. Compute theta
    # 3. Project it to P0 elements
    # 4. Multiply by psi-mass
    time_rhs[-dof_psi:] += M_psi @ project_psi_to_fe(
        theta(proj_psi @ sol[-1][-dof_psi:])
    )

    # Solution at the previous iteration (k=0 corresponds to the solution at the previous time step)
    prev = sol[-1]
    current = None

    # Non-linear solver
    for k in np.arange(K):
        # Actual rhs
        rhs = time_rhs.copy()

        # \Theta^{n+1}_k, same steps as \Theta^n
        rhs[-dof_psi:] -= M_psi @ project_psi_to_fe(theta(proj_psi @ prev[-dof_psi:]))

        # L-term
        rhs[-dof_psi:] += L * M_psi @ prev[-dof_psi:]

        # Set the conductivity tensor in data (the actual saved tensor does not matter at this stage)
        pp.initialize_data(
            subdomain,
            data,
            key,
            {
                "second_order_tensor": pp.SecondOrderTensor(
                    conductivity(proj_psi @ prev[-dof_psi:])
                ),
            },
        )

        Mass_u = pg.face_mass(mdg, RT0)

        # Assemble the system to be solved at time n and interation k
        spp = sps.block_array([[Mass_u, B.T], [-dt * B, L * M_psi]], format="csc")

        # Prepare the linear solver
        ls = pg.LinearSystem(spp, rhs)

        # Fix the essential boundary conditions
        ls.flag_ess_bc(np.hstack(bc_essential), np.zeros(dof_q + dof_psi))

        # Solve the system
        current = ls.solve()

        # Check if we have reached convergence
        rel_err_psi = np.sqrt(np.sum(np.power(current[-dof_psi:] - prev[-dof_psi:], 2)))
        abs_err_prev = np.sqrt(np.sum(np.power(prev[-dof_psi:], 2)))

        # Log message with error and current iteration
        print(
            "Iteration #" + str(k + 1) + ", error L2 relative psi: " + str(rel_err_psi)
        )

        if rel_err_psi > abs_tol + rel_tol * abs_err_prev:
            prev = current.copy()
        else:
            break

    sol.append(current)
    export_solution(saver, current_sol=sol[-1], num_step=(n + 1))

saver.write_pvd([n * dt for n in np.arange(num_steps + 1)])

Time 0.02083
Iteration #1, error L2 relative psi: 0.04475035785359601
Iteration #2, error L2 relative psi: 0.004959237083634829
Iteration #3, error L2 relative psi: 0.00013141900633286294
Iteration #4, error L2 relative psi: 1.1796524616887123e-05
Iteration #5, error L2 relative psi: 1.0030186277703706e-06
Time 0.04167
Iteration #1, error L2 relative psi: 0.04058890823654569
Iteration #2, error L2 relative psi: 0.008831518742316576
Iteration #3, error L2 relative psi: 0.00036845498951560996
Iteration #4, error L2 relative psi: 3.955655314513511e-05
Iteration #5, error L2 relative psi: 7.504562341774411e-06
Iteration #6, error L2 relative psi: 4.3261642805091825e-07
Time 0.0625
Iteration #1, error L2 relative psi: 0.03498375700537033
Iteration #2, error L2 relative psi: 0.005909431697391049
Iteration #3, error L2 relative psi: 0.0004201269208700117
Iteration #4, error L2 relative psi: 3.566701373165286e-05
Iteration #5, error L2 relative psi: 3.6808344906061575e-06
Iteration #6, error L

A representation of the computed solution is given below, where the cells are colored with $p$ and the arrows are the $q$. <br>
![](fig/richards.png)